# Island Surface Defect Formation on TiN(001) Surface

## 0. Introduction

This tutorial demonstrates the process of creating a rectangular island defect on a slab. 


> **D. G. Sangiovanni, A. B. Mei, D. Edström, L. Hultman, V. Chirita, I. Petrov, and J. E. Greene**, "Effects of surface vibrations on interlayer mass transport: Ab initio molecular dynamics investigation of Ti adatom descent pathways and rates from TiN/TiN(001) islands", Physical Review B, 2018. [DOI: 10.1103/PhysRevB.97.035406](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.97.035406).
> 

![Fig 2.](https://i.imgur.com/wgh5QRf.png)

In this notebook we will reproduce material from FIG. 2 a)


## 1. Prepare the Environment
### 1.1. Set up defects parameters 



In [ ]:
AUTO_ADD_VACUUM = True
VACUUM_THICKNESS = 15.0
NUMBER_OF_ADDED_LAYERS = 0.5

SHAPE_PARAMETERS = {
    'shape': 'box',
    'min_coordinate': [0.25, 0.2, 0],
    'max_coordinate': [0.65, 0.6, 1]
}

# Create slab with default parameters if imported material is not a slab
DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 3,
    "vacuum": 0.0,
    "use_orthogonal_z": True,
    "xy_supercell_matrix": [[10, 0], [0, 10]]
}

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

material = Material(Materials.get_by_name_first_match("TiN"))

### 1.4. Preview Slab or create a new one

In [ ]:
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration, get_terminations
from utils.visualize import visualize_materials as visualize

slab = material
if not slab.metadata or slab.metadata["build"]["configuration"]["type"] != SlabConfiguration.__name__:
    print("The material is not a slab. Creating a new slab...")
    slab_config = SlabConfiguration(
        bulk=material,
        miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
        thickness=DEFAULT_SLAB_PARAMETERS["thickness"],
        vacuum=DEFAULT_SLAB_PARAMETERS["vacuum"],
        use_orthogonal_z=DEFAULT_SLAB_PARAMETERS["use_orthogonal_z"],
        xy_supercell_matrix=DEFAULT_SLAB_PARAMETERS["xy_supercell_matrix"]
    )
    terminations = get_terminations(slab_config)
    print(f"Available terminations: {terminations}")
    slab = create_slab(slab_config, terminations[0])

visualize([{"material": slab, "title": "Original material"}])
visualize([{"material": slab, "title": "Original material"}], rotation="-90x")

## 2. Create the Target Material
### 2.1. Initialize the defect configuration and builder

In [ ]:
from mat3ra.made.tools.utils.coordinate import (
    BoxCoordinateCondition,
)
from mat3ra.made.tools.build.defect import IslandSlabDefectConfiguration

coordinate_condition = BoxCoordinateCondition(
    min_coordinate=SHAPE_PARAMETERS['min_coordinate'],
    max_coordinate=SHAPE_PARAMETERS['max_coordinate']
)

island_config = IslandSlabDefectConfiguration(
    crystal=slab,
    condition=coordinate_condition,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
    use_cartesian_coordinates=False,
)

from mat3ra.made.tools.build.defect import IslandSlabDefectBuilder, SlabDefectBuilderParameters

params = SlabDefectBuilderParameters(
    auto_add_vacuum=AUTO_ADD_VACUUM,
    vacuum_thickness=VACUUM_THICKNESS,
)
builder = IslandSlabDefectBuilder(build_parameters=params)

### 2.2. Create the island

In [ ]:
slab_with_island = builder.get_material(island_config)

## 3. Visualize the Result(s)

In [ ]:
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({SHAPE_PARAMETERS['shape']})"}],
          rotation="-90x")

visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": f"Material with Island Defect ({SHAPE_PARAMETERS['shape']})"}])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file

download_content_to_file(slab_with_island, "TiN_slab_with_island.json")